In [146]:
import os
import sys 
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../../latent_analysis/'))
sys.path.append(os.path.abspath('../../dPCA/python/'))
import LTransform as LT
from get_utils import get_hidden, get_dir
from pathlib import Path
import numpy as np
import torch as th
import plot as plot
import matplotlib.pyplot as plt

In [140]:
# Load hidden
folder_name = 'loss17'
model_num = 0
model_name = "model{:02d}".format(model_num)

ff_coef = [8,8,8,8]
phase = ['NF1','FF1','NF2','FF2']

data = get_hidden(folder_name,model_name,phase,ff_coef,demean=False)


# Apply SVD
n_muscle = 6
n_latent = 6

weight_file,_,_= get_dir(folder_name,model_name,phase[0])
W = th.load(weight_file)['fc.weight'].numpy()
print(W.shape)
U, S, Vh = np.linalg.svd(W, full_matrices=True)
V = Vh.T
P = V[:,:n_muscle] # output potent
N = V[:,n_muscle:] # output null


# Apply PCA
Data_prep =  {'x_pot_ld':[],'x_null_ld':[],'x_pot':[],'x_null':[]}
for i in range(4):
    X = data[i]
    dims = X.shape

    X_pot = X@P
    X_null = X@N

    X_pot = np.reshape(X_pot,newshape=dims[:-1]+(n_muscle,))
    X_null = np.reshape(X_null,newshape=dims[:-1]+(dims[-1]-n_muscle,))

    if i==0:
        T_p = LT.Transform(num_latent=n_latent)
        T_p.fit(X_pot,method='PCA')

        T_l = LT.Transform(num_latent=n_latent)
        T_l.fit(X_null,method='PCA')

    x_pot_ld = T_p.transform(X_pot,ensure_orthogonality=True)
    x_null_ld = T_l.transform(X_null,ensure_orthogonality=True)

    Data_prep['x_pot_ld'].append(x_pot_ld)
    Data_prep['x_null_ld'].append(x_null_ld)
    Data_prep['x_pot'].append(X_pot)
    Data_prep['x_null'].append(X_null)

(6, 128)


In [141]:
#%matplotlib widget
%matplotlib qt
# ['x', 'o', '*', 's']
latent = [0,1,2]
dim=2
which_times = np.arange(10)
which_times =[10]
plot_scatter=1

if dim==2:
    fig,ax = plt.subplots(1,2,figsize=(10,5))
else:
    fig,ax = plt.subplots(1,2,figsize=(10,5),subplot_kw={'projection':'3d'})

plot.plot_traj(ax[0],Data_prep['x_null_ld'],which_times=which_times,plot_scatter=plot_scatter,dim=dim,which_latent=latent)
ax[0].set_title('null')

plot.plot_traj(ax[1],Data_prep['x_pot_ld'],which_times=which_times,plot_scatter=plot_scatter,dim=dim,which_latent=latent)
ax[1].set_title('potent')

plt.tight_layout()
plt.show()

In [145]:
# 0-NF1, 1-FF1, 2-NF2, 3-FF2
# 0-(0,1) NF1-FF1 
# 1-(0,2) NF1-NF2 X
# 2-(0,3) NF1-FF2 X
# 3-(1,2) FF1-NF2 X
# 4-(1,3) FF1-FF2 X
# 5-(2,3) NF2-FF2

from scipy.spatial.distance import pdist

D = np.stack(Data_prep['x_pot'])
dist = np.zeros((8,100,6))
for i in range(8):
    for j in range(100):
        dist[i,j,:] = pdist(D[:,i,j,:])

dist = np.mean(dist,axis=0)

x = np.linspace(0, 1, dist.shape[0])
color_list = ['blue','orange','orange','orange']
style = ['-','-','-','--']
fig,ax = plt.subplots(1,1,figsize=(10,5))
label = ['NF1-FF1','NF1-NF2','NF1-FF2','FF1-NF2','FF1-FF2','NF2-FF2']

for i,d in enumerate([0,3]):
    ax.plot(x,dist[:,d],color=color_list[i],linestyle=style[i],label=label[d])
ax.legend()
plt.tight_layout()
plt.show()